In [3]:

import os
import time
import duckdb
import geopandas as gpd
from fiona import listlayers
%load_ext sql

import logging
#set up logging 
logging.basicConfig(level=logging.DEBUG)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

tempdir = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\MarxanInputs\Contiguous'
gdb = os.path.join(tempdir, "ContiguousHabitat_2025-06-25.gdb")
tempgdb = os.path.join(tempdir, "TEMP_ContiguousHabitat.gdb")

studyArea = r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\WMB_Study_Area_2024_07_30\WMB_Study_Area_2024_07_30.shp"
privateLand = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\ownership\iflb_own_studyArea.shp'
vri_gdb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance products for BRFN - Copy\data\BRFN_FOREST_DSTRB.gdb'
wmb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Project\WMB_Planning.gdb\StudyArea_Erase' # Note that this data is the study area split by the closest WMB

disturbance_dir = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance'

workspace = r'C:\Users\NROSS\OneDrive - Government of BC\Documents\Projects\WMBTesting\wmb-duckdb'
work_db = os.path.join(workspace, "contiguous-habitat-work.db")

ModuleNotFoundError: No module named 'duckdb'

In [2]:
import os
import logging
#set up logging 
logging.basicConfig(level=logging.DEBUG)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

gdb = r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\ContiguousHabitat_2025-06-25.gdb"
tempgdb = r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\TEMP_ContiguousHabitat.gdb"

studyArea = r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\WMB_Study_Area_2024_07_30\WMB_Study_Area_2024_07_30.shp"
privateLand = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\ownership\iflb_own_studyArea.shp'
vri_gdb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance products for BRFN - Copy\data\BRFN_FOREST_DSTRB.gdb'

disturbance_dir = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance'

workspace = r'C:\Users\NROSS\OneDrive - Government of BC\Documents\Projects\WMBTesting\wmb-duckdb'
work_db = os.path.join(workspace, "contiguous-habitat-work.db")

In [2]:
# connect to duckdb
con = duckdb.connect(work_db)
con.execute("INSTALL spatial; LOAD spatial;")

In [3]:
# Read study area and private land

study_gdf = gpd.read_file(studyArea)
study_gdf['wkt'] = study_gdf['geometry'].to_wkt()
study_gdf = study_gdf.drop(columns='geometry')
con.sql("CREATE TABLE IF NOT EXISTS studyarea AS (SELECT *,  ST_GeomFromText(wkt) as GEOM FROM study_gdf)")

private_gdf = gpd.read_file(privateLand)
private_gdf['wkt'] = private_gdf['geometry'].to_wkt()
private_gdf = private_gdf.drop(columns='geometry')
con.sql("CREATE TABLE IF NOT EXISTS privateland AS (SELECT *,  ST_GeomFromText(wkt) as GEOM FROM private_gdf)")

In [ ]:
def read_fc_to_duckdb(gdb, layer, src_name, clause, duckdbTable, broadBuffer, refinedBuffer):
    """Reads feature class or shapefile to geopandas, removes all fields except the layer name, adds appropriate buffer fields, converts to duckdb and inserts into a table

    Args:
        gdb (str): path to geodatabase
        layer (str): layer (feature class) name. Leave None for shapefile or to pick the first layer
        src_name (str): Name to list in the source column in the duckdb table
        clause (str): SQL clause when reading the input data
        duckdbTable (str): Name of existing table in duckdb
        broadBuffer (int): Buffer distance for broad contiguous habitat. Set to None to ignore for this category.
        refinedBuffer (int): Buffer distance for refined contiguous habitat
    """
    info(f"Reading {src_name} to geopandas with clause {clause}")
    gdf = gpd.read_file(gdb, layer=layer, where=clause)
    info(f'\tContains {len(gdf)} records')
    gdf['wkt'] = gdf['geometry'].to_wkt()
    gdf = gdf.drop(columns='geometry')
    gdf['source'] = src_name
    gdf['broadbuffer'] = broadBuffer
    gdf['refinedbuffer'] = refinedBuffer
    
    info(f"\t inserting into {duckdbTable}")
    con.sql(f"INSERT INTO {duckdbTable} BY NAME (SELECT ST_GeomFromText(gdf.wkt) as GEOM, source, broadbuffer, refinedbuffer FROM gdf JOIN studyarea s ON ST_Intersects(ST_GeomFromText(gdf.wkt), s.GEOM))")
    del gdf

In [ ]:
# Create all_dist table and add W2W layers

con.sql("DROP TABLE IF EXISTS all_dist")
con.sql("CREATE SEQUENCE IF NOT EXISTS seq_distid START 1;")
con.sql("CREATE TABLE all_dist (id INTEGER PRIMARY KEY DEFAULT nextval('seq_distid'), geom GEOMETRY, source VARCHAR, broadbuffer FLOAT, refinedbuffer FLOAT)")

w2w_gdb = os.path.join(disturbance_dir, 'Wall_To_Wall_2025_06_05.gdb')

# list disturbance buffer categories
broad_standard = 50
refined_standard = 100
broad_noisy = 50
refined_noisy = 500
refined_seismic_mech = 50
refined_seismic_legacy = 100

def get_w2w_buffers_dict(fc):
    
    outDict = {}
        
    if fc in ['RSEA_Seismic', 'RSEA_Fire', 'RSEA_Cutblocks']: # add any layers to ignore to this list
        outDict['broadbuffer'] = None
        outDict['refinedbuffer'] = None
    elif fc == "RSEA_AG":
        outDict['broadbuffer'] = 0
        outDict['refinedbuffer'] = 0
    elif fc == 'RSEA_Rail':
        outDict['broadbuffer'] = broad_noisy
        outDict['refinedbuffer'] = refined_noisy
        # the other "noisy" datasets are outside the w2w
        
    elif "Seismic_" in fc:
        outDict['broadbuffer'] = None
        if fc =='Seismic_BCER_Legacy2D_WMBcleaned_poly':
            outDict['refinedbuffer'] = refined_seismic_legacy
        elif fc in ['Seismic_BCER_1996_2004_WMBcleaned_poly', 'Seismic_BCER_2002_2006_WMBcleaned_poly', 'Seismic_BCER_Permitted_WMBcleaned_poly']:
            outDict['refinedbuffer'] = refined_seismic_mech
            
            # Note - these need all lines except CutType = "Mech" removed
            outDict['clause'] = "CutType = 'Mech'"
            
    else: # set all others to the standard buffer distance
        outDict['broadbuffer'] = broad_standard
        outDict['refinedbuffer'] = refined_standard
    
    if 'clause' not in outDict:
        outDict['clause'] = ''
    
    return outDict

for fc in listlayers(w2w_gdb):
    # Ignore the "line" features and a few RSEA features
    if '_line' in fc or fc in ['RSEA_Seismic', 'RSEA_Fire', 'RSEA_Cutblocks']:
        continue
    
    # Get buffer dictionary
    bufferDict = get_w2w_buffers_dict(fc)
    
    # insert into duckdb 'all_dist' table
    read_fc_to_duckdb(w2w_gdb, fc, fc, bufferDict['clause'], 'all_dist', bufferDict['broadbuffer'], bufferDict['refinedbuffer'])
    


DEBUG:fiona._env:GDAL_DATA found in environment.


DEBUG:fiona._env:PROJ_DATA found in environment.
INFO:root:Reading Applications_inReview_19Sept2024 to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading FTML_WOODLOTS_AandB_activeonly_deferredRemoved to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading PASR_Assoc_Ancillary_poly to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading PASR_Pipeline_poly to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading PASR_Road_poly to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading PASR_WellFacility_Pads_poly to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading Pipeline_ROWs_Infill_2024_12_04 to geopandas with clause 
c:\Users\NROSS\.conda\envs\pyduckdb\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by settin

In [ ]:
# Add VRI cutblocks
read_fc_to_duckdb(
    vri_gdb, 
    layer = 'merged', 
    src_name = 'VRI Cutblocks',
    clause = "DSTRB_HIST LIKE '%CUT%' And MRSRD_Y > 1984", 
    duckdbTable='all_dist', 
    broadBuffer = broad_standard, 
    refinedBuffer = refined_standard
    )

# Add noisy disturbances
noisy_dist_dir = os.path.join(disturbance_dir, 'NoisyDisturbances', 'Updates_forNorth')
for shp in ['Compressors_Plants_wShape.shp', 'MainRoads_2024_12_04_Footprint.shp']:
    read_fc_to_duckdb(
        os.path.join(noisy_dist_dir, shp), 
        layer = None, 
        src_name = shp,
        clause = '', 
        duckdbTable='all_dist', 
        broadBuffer = broad_noisy, 
        refinedBuffer = refined_noisy)

INFO:root:Reading VRI Cutblocks to geopandas with clause DSTRB_HIST LIKE '%CUT%' And MRSRD_Y > 1984
INFO:root:	 inserting into all_dist
INFO:root:Reading Compressors_Plants_wShape.shp to geopandas with clause 
INFO:root:	 inserting into all_dist
INFO:root:Reading MainRoads_2024_12_04_Footprint.shp to geopandas with clause 
INFO:root:	 inserting into all_dist


In [ ]:
# Setup sql magic and show current tables
%sql con --alias duckdb
%sql select * from information_schema.tables

Running query in 'duckdb'

table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
contiguous-habitat-work,main,all_dist,BASE TABLE,None,None,None,None,None,YES,NO,None,None
contiguous-habitat-work,main,privateland,BASE TABLE,None,None,None,None,None,YES,NO,None,None
contiguous-habitat-work,main,studyarea,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [9]:
%%sql
DROP TABLE IF EXISTS dist_buffer_broad;
DROP TABLE IF EXISTS dist_buffer_refined;

Running query in 'duckdb'

Success


In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS dist_buffer_broad AS (
    SELECT id, source, ST_Simplify(ST_Buffer(geom, broadbuffer), 10) as geom FROM all_dist WHERE broadbuffer IS NOT NULL
);

CREATE TABLE IF NOT EXISTS dist_buffer_refined AS (
    SELECT id, source, ST_Simplify(ST_Buffer(geom, refinedbuffer), 10) as geom FROM all_dist WHERE refinedbuffer IS NOT NULL
);


CREATE TABLE IF NOT EXISTS dist_buffer_refined_50 AS (
    SELECT id, source, ST_Simplify(ST_Buffer(geom, 50), 10) as geom FROM all_dist WHERE refinedbuffer IS NOT NULL
);

Running query in 'duckdb'

Count
1327204


In [ ]:
def duckdb_to_gdb(table_name):
    gdf = con.sql(f"SELECT id, source, ST_AsText(GEOM) as wkt from {table_name}").to_df()
    gdf['geometry'] = gpd.GeoSeries.from_wkt(gdf['geometry'])
    gdf = gpd.GeoDataFrame(gdf)
    gdf.to_file(tempgdb, layer=table_name)

duckdb_to_gdb('dist_buffer_broad')
duckdb_to_gdb('dist_buffer_refined')
duckdb_to_gdb('dist_buffer_refined_50')

In [14]:
con.close()

In [ ]:
# switch to arcpy for the last part due to duckdb issues with processing this many verticies
import arcpy
import pandas as pd
import os

tempdir = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\MarxanInputs\Contiguous'
gdb = os.path.join(tempdir, "ContiguousHabitat_20250625.gdb")
tempgdb = os.path.join(tempdir, "TEMP_ContiguousHabitat.gdb")

studyArea = r"\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\WMB_Study_Area_2024_07_30\WMB_Study_Area_2024_07_30.shp"
privateLand = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\ownership\private_40_41_studyArea.shp'
vri_gdb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance products for BRFN - Copy\data\BRFN_FOREST_DSTRB.gdb'
wmb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Project\WMB_Planning.gdb\StudyArea_Erase' # Note that this data is the study area split by the closest WMB
buffer_geopackage = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\MarxanInputs\Contiguous\contiguous_2025-06-26.gpkg'

disturbance_dir = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance'

arcpy.env.workspace = tempgdb
arcpy.env.overwriteOutput = True

# for distType in ['broad', 'refined']:
for distType in ['refined']:
    # erase the buffered disturbance from the study area
    print(f"erasing {distType} in {tempgdb}")
    arcpy.analysis.Erase(
        in_features=studyArea,
        erase_features=f"dist_buffer_{distType}_dissolve",
        out_feature_class=f"interior_{distType}_1",
        cluster_tolerance=None
    )

    # erase the private land from this
    print(f"erasing private land from {distType}")
    arcpy.analysis.Erase(
        in_features=f"interior_{distType}_1",
        erase_features=privateLand,
        out_feature_class=f"interior_{distType}_noPrivate",
        cluster_tolerance=None
    )
    # explode to singlepart
    print(f"explode {distType}")
    arcpy.management.MultipartToSinglepart(
        in_features=f"interior_{distType}_noPrivate",
        out_feature_class=f"interior_{distType}_noPrivate_exploded"
    )

    # Calculate hectares
    print(f"calculate area {distType}")
    arcpy.management.CalculateField(
        in_table=f"interior_{distType}_noPrivate_exploded",
        field="InteriorAreaHa",
        field_type="DOUBLE",
        expression="!Shape_Area!/10000"
    )
    # Intersect with WMB
    print(f"WMB intersect {distType}")
    arcpy.analysis.Identity(
        in_features=f"interior_{distType}_noPrivate_exploded",
        identity_features=wmb,
        out_feature_class=f"interior_{distType}_noPrivate_exploded_byWMB",
        join_attributes="ALL",
        cluster_tolerance=None,
        relationship="NO_RELATIONSHIPS"
    )

    # get the Threshold values from Pandas
    # Convert feature class to NumPy array then pandas dataframe
    numpy_array = arcpy.da.FeatureClassToNumPyArray(f"interior_{distType}_noPrivate_exploded_byWMB", ['OBJECTID', 'WMB', 'Inner', 'Shape_Area'])
    df = pd.DataFrame(numpy_array)

    df['AreaHa'] = df['Shape_Area']/10000
    sorteddf = df.loc[df['Inner']=='Inner'].sort_values('AreaHa', ascending=False)
    sorteddf['Cumul'] = sorteddf.groupby(["WMB"])['AreaHa'].transform(pd.Series.cumsum) 
    # Get the sum of all patches by WMB
    sorteddf['WMBInteriorTotal'] = sorteddf.groupby(["WMB"])['AreaHa'].transform(pd.Series.sum)
    # Get percentage of each cumulative step of the entire df
    sorteddf['Interior_Cumulative_Entire_pct'] = sorteddf['Cumul'] / sorteddf['WMBInteriorTotal']

    # Create summary tables showing the last (meaning maximum in each group) polygon area and OBJECTID before reaching the 25% and 50% values for the entire area.
    p25 = sorteddf.loc[sorteddf['Interior_Cumulative_Entire_pct'] <= 0.25][["WMB", 'AreaHa', 'OBJECTID']].groupby(["WMB"]).last()
    p50 = sorteddf.loc[sorteddf['Interior_Cumulative_Entire_pct'] <= 0.5][["WMB",'AreaHa', 'OBJECTID', 'Cumul']].groupby(["WMB"]).last()

    # Join these two together into one threshold dataframe and display
    thresh_df = p25.join(p50, lsuffix="_25", rsuffix="_50").reset_index()
    print(thresh_df)
    df = df.merge(thresh_df[['WMB', 'AreaHa_25', 'AreaHa_50']], on='WMB')
    df.loc[df['AreaHa'] >= df['AreaHa_50'], 'threshold'] = 50.0
    df.loc[df['AreaHa'] >= df['AreaHa_25'], 'threshold'] = 25.0

    # Export this table to a csv and join the field back on to the Interior area
    df[['OBJECTID', 'WMB', 'threshold']].to_csv(os.path.join(tempdir, f'interior_{distType}_threshold.csv'))

    print("Joining threshold field to interior")
    # JOIN FIELD to the interior on OBJECTID
    arcpy.management.JoinField(
        in_data=f"interior_{distType}_noPrivate_exploded_byWMB",
        in_field="OBJECTID",
        join_table=os.path.join(tempdir, f'interior_{distType}_threshold.csv'),
        join_field="OBJECTID",
        fields="threshold",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )

    # export this as the Interior layer
    arcpy.conversion.FeatureClassToFeatureClass(
        f"interior_{distType}_noPrivate_exploded_byWMB",
        out_path=gdb, out_name=f"interior_{distType}"
    )
    if distType == 'broad':

        # Buffer out again
        print("Buffering again")
        arcpy.analysis.Buffer(
            in_features=f"interior_{distType}_noPrivate_exploded_byWMB",
            out_feature_class=f"contiguous_{distType}_buffered",
            buffer_distance_or_field=50
        )

        # erase private again
        print("erase private again")
        arcpy.analysis.Erase(
            in_features=f"contiguous_{distType}_buffered",
            erase_features=privateLand,
            out_feature_class=os.path.join(gdb, f"Contiguous_{distType}"),
            cluster_tolerance=None
        )

    else:
        # Take the Refined dist buffered by 50m
        # Erase this from the study area and explode
        print(f"Erasing disturbance refined from study area")
        arcpy.analysis.Erase(
            in_features=studyArea,
            erase_features=f"dist_buffer_{distType}_50_dissolve",
            out_feature_class=f"interior_{distType}_50",
            cluster_tolerance=None
        )
        # explode to singlepart
        print("explode")
        arcpy.management.MultipartToSinglepart(
            in_features=f"interior_{distType}_50",
            out_feature_class=f"interior_{distType}_50_exploded"
        )

        # buffer each of these by 50m 
        print("Buffer these by 50")
        arcpy.analysis.Buffer(
            in_features=f"interior_{distType}_50_exploded",
            out_feature_class=f"contiguous_{distType}_1",
            buffer_distance_or_field=50
        )

        # erase private again
        print("erase private again")
        arcpy.analysis.Erase(
            in_features=f"contiguous_{distType}_1",
            erase_features=privateLand,
            out_feature_class=f"contiguous_{distType}_2",
            cluster_tolerance=None
        )
        # Use summarize within to get the sum, mean, max and count of refined interior are within
        print("summarize refined interior within these areas")
        arcpy.management.CalculateField(f"interior_{distType}_noPrivate_exploded_byWMB", "threshold_int", "int(!threshold!)", field_type = "SHORT")
        arcpy.analysis.SummarizeWithin(
            in_polygons=f"contiguous_{distType}_2",
            in_sum_features=f"interior_{distType}_noPrivate_exploded_byWMB",
            out_feature_class=os.path.join(gdb, f"Contiguous_{distType}"),
            keep_all_polygons="KEEP_ALL",
            sum_fields="InteriorAreaHa Max;InteriorAreaHa Mean;InteriorAreaHa Sum; threshold_int Min",
            sum_shape="NO_SHAPE_SUM",
            shape_unit="HECTARES",
            group_field=None,
            add_min_maj="NO_MIN_MAJ",
            add_group_percent="NO_PERCENT",
            out_group_table=None
        )

In [20]:
# buffer each of these by 50m 
print("Buffer these by 50")
# arcpy.analysis.Buffer(
#     in_features=f"interior_{distType}_50_exploded",
#     out_feature_class=f"contiguous_{distType}_1",
#     buffer_distance_or_field=50
# )


# Use summarize within to get the sum, mean, max and count of refined interior are within
print("summarize refined interior within these areas")
arcpy.analysis.SummarizeWithin(
    in_polygons=f"contiguous_{distType}_1",
    in_sum_features=f"interior_{distType}_noPrivate_exploded_byWMB",
    out_feature_class=os.path.join(gdb, f"Contiguous_{distType}"),
    keep_all_polygons="KEEP_ALL",
    sum_fields="InteriorAreaHa Max;InteriorAreaHa Mean;InteriorAreaHa Sum; threshold_int Min",
    sum_shape="NO_SHAPE_SUM",
    shape_unit="HECTARES",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

Buffer these by 50
summarize refined interior within these areas


<Result '\\\\spatialfiles.bcgov\\work\\srm\\fsj\\Workarea\\nross\\WMBPlanning\\Data\\MarxanInputs\\Contiguous\\ContiguousHabitat_20250625.gdb\\Contiguous_refined'>